#Preset

In [ ]:
#@title CARGA DE DATOS
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/P_IA')
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
dfo = pd.read_csv("new_data_urls.csv")
dfb = pd.read_csv("df2.csv")
import joblib

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Primary TPU Activation

import tensorflow as tf
import os
import tensorflow_datasets as tfds


try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  print('Running on TPU ', tpu.master())
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
print("All devices: ", tf.config.list_logical_devices('TPU'))

Running on TPU  
All devices:  [LogicalDevice(name='/device:TPU:0', device_type='TPU'), LogicalDevice(name='/device:TPU:1', device_type='TPU'), LogicalDevice(name='/device:TPU:2', device_type='TPU'), LogicalDevice(name='/device:TPU:3', device_type='TPU'), LogicalDevice(name='/device:TPU:4', device_type='TPU'), LogicalDevice(name='/device:TPU:5', device_type='TPU'), LogicalDevice(name='/device:TPU:6', device_type='TPU'), LogicalDevice(name='/device:TPU:7', device_type='TPU')]


In [ ]:
#@title Funciones
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
def evaluate_model(model, X_test, y_test):
  y_pred_probs = model.predict(X_test)
  y_pred = (y_pred_probs > 0.5).astype(int)

  tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

  try:
    tpr = tp / (tp + fn)
  except ZeroDivisionError:
    tpr = 0.0

  try:
    tnr = tn / (tn + fp)
  except ZeroDivisionError:
    tnr = 0.0

  return tpr, tnr


In [ ]:
from re import U
#@title LIMIEZA Y ANALISIS DE DATOS
from urllib.parse import urlparse
def AnalizarTxt(df):
  import pandas as pd
  from urllib.parse import urlparse
  urls = df["url"].astype(str)  # Convert 'url' column to string type
  types = df.values[:, 1]
  types = types.astype('bool')

  # Apply urlparse to each URL individually
  parsed_urls = urls.apply(urlparse)

  puntos = {}
  guionb = {}
  slashb = {}
  Tamaño = {}
  guion = {}
  slash = {}
  commilla = {}
  almoadilla = {}
  dolar = {}
  porcentaje = {}
  IsIP = {} # Initialize IsIP

  for i in ["scheme", "netloc", "path", "params", "query", "fragment"]:
    puntos[i] = parsed_urls.apply(lambda x: getattr(x, i).count("."))
    guion[i] = parsed_urls.apply(lambda x: getattr(x, i).count("-"))
    slash[i] = parsed_urls.apply(lambda x: getattr(x, i).count("/"))
    Tamaño[i] = parsed_urls.apply(lambda x: len(getattr(x, i)))
    guionb[i] = parsed_urls.apply(lambda x: "_" in getattr(x, i))
    slashb[i] = parsed_urls.apply(lambda x: "\\" in getattr(x, i))
    commilla[i] = parsed_urls.apply(lambda x: "\"" in getattr(x, i) or "\'" in getattr(x, i))
    almoadilla[i] = parsed_urls.apply(lambda x: "#" in getattr(x, i))
    dolar[i] = parsed_urls.apply(lambda x: "$" in getattr(x, i))
    porcentaje[i] = parsed_urls.apply(lambda x: "%" in getattr(x, i))
  IsIP = parsed_urls.apply(lambda x: r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}" in x.netloc)


  DFnew=pd.DataFrame({
        #'url': url,
        "puntosscheme": puntos["scheme"],
        "guionsscheme": guion["scheme"],
        "slashscheme": slash["scheme"],
        "Tamañosscheme": Tamaño["scheme"],
        "guionbscheme": guionb["scheme"],
        "slashbscheme": slashb["scheme"],
        "commillascheme": commilla["scheme"],
        "almoadillascheme": almoadilla["scheme"],
        "dolarsscheme": dolar["scheme"],
        "porcentajescheme": porcentaje["scheme"],
        "puntosnetoloc": puntos["netloc"],
        "guionnetoloc": guion["netloc"],
        "slashnetoloc": slash["netloc"],
        "Tamañonetoloc": Tamaño["netloc"],
        "guionbnetoloc": guionb["netloc"],
        "slashbnetoloc": slashb["netloc"],
        "commillasnetoloc": commilla["netloc"],
        "almoadillanetoloc": almoadilla["netloc"],
        "dolarsnetoloc": dolar["netloc"],
        "porcentajenetoloc": porcentaje["netloc"],
        "puntospath": puntos["path"],
        "guionpath": guion["path"],
        "slashpath": slash["path"],
        "Tamañospath": Tamaño["path"],
        "guionbpath": guionb["path"],
        "slashbpath": slashb["path"],
        "commillaspath": commilla["path"],
        "almoadillaspath": almoadilla["path"],
        "dolarspath": dolar["path"],
        "porcentajepath": porcentaje["path"],
        "puntosparams": puntos["params"],
        "guionparams": guion["params"],
        "slashparams": slash["params"],
        "Tamañosparams": Tamaño["params"],
        "guionbparams": guionb["params"],
        "slashbparams": slashb["params"],
        "commillasparams": commilla["params"],
        "almoadillasparams": almoadilla["params"],
        "dolarsparams": dolar["params"],
        "porcentajeparams": porcentaje["params"],
        "puntosquery": puntos["query"],
        "guionquery": guion["query"],
        "slashquery": slash["query"],
        "Tamañosquery": Tamaño["query"],
        "guionbquery": guionb["query"],
        "slashbquery": slashb["query"],
        "commillasquery": commilla["query"],
        "almoadillasquery": almoadilla["query"],
        "dolarquery": dolar["query"],
        "porcentajesquery": porcentaje["query"],
        "puntosfragment": puntos["fragment"],
        "guionfragment": guion["fragment"],
        "slashfragment": slash["fragment"],
        "Tamañosfragment": Tamaño["fragment"],
        "guionbfragment": guionb["fragment"],
        "slashbfragment": slashb["fragment"],
        "commillasfragment": commilla["fragment"],
        "almoadillasfragment": almoadilla["fragment"],
        "dolarfragment": dolar["fragment"],
        "porcentajesfragment": porcentaje["fragment"],
        'IsIP': IsIP,
        'types': types

    })
  return DFnew

df = AnalizarTxt(dfb)
X = df.drop(columns=["types"]).values
y = df["types"].values
unique_values = len(np.unique(y))
encoder = LabelEncoder()

for i in range(X.shape[1]):
    if isinstance(X[0, i], str):
        X[:, i] = encoder.fit_transform(X[:, i])
    else:
        X[:, i] = X[:, i].astype(np.float32)
X = X.astype(np.float32)
y = y.astype(np.int32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1, random_state=21)

In [ ]:
dfb.head()

,url,type
0,br-icloud.com.br,1
1,http://buzzfil.net/m/show-art/ils-etaient-loin...,0
2,signin.eby.de.zukruygxctzmmqi.civpro.co.za,1
3,http://hollywoodlife.com/2014/05/01/rihanna-ih...,0
4,http://www.marketingbyinternet.com/mo/e56508df...,1


In [ ]:
df.head()

,puntosscheme,guionsscheme,slashscheme,Tamañosscheme,guionbscheme,slashbscheme,commillascheme,almoadillascheme,dolarsscheme,porcentajescheme,...,slashfragment,Tamañosfragment,guionbfragment,slashbfragment,commillasfragment,almoadillasfragment,dolarfragment,porcentajesfragment,IsIP,types
0,0,0,0,0,False,False,False,False,False,False,...,0,0,False,False,False,False,False,False,False,True
1,0,0,0,4,False,False,False,False,False,False,...,0,0,False,False,False,False,False,False,False,False
2,0,0,0,0,False,False,False,False,False,False,...,0,0,False,False,False,False,False,False,False,True
3,0,0,0,4,False,False,False,False,False,False,...,0,0,False,False,False,False,False,False,False,False
4,0,0,0,4,False,False,False,False,False,False,...,0,0,False,False,False,False,False,False,False,True


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(77994, 61)
(77994,)
(1, 61)
(1,)


In [ ]:
loaded_models = []
import joblib
import os
from tensorflow import keras

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define the path to your models
model_path = '/content/drive/My Drive/P_IA'

# Iterate through the files in the model directory
for filename in os.listdir(model_path):
    if filename.endswith('.joblib'):
        model_filepath = os.path.join(model_path, filename)
        print(f"Loading model from: {model_filepath}")
        try:
            loaded_model = joblib.load(model_filepath)
            print(f"Successfully loaded {filename}")
            # Now you can use the loaded model
            # ... your code to use the loaded model ...
            loaded_models.append(loaded_model)
        except Exception as e:
            print(f"Error loading {filename}: {e}")

    elif filename.endswith('.h5'):
        model_filepath = os.path.join(model_path, filename)
        print(f"Loading model from: {model_filepath}")
        try:
            loaded_model = keras.models.load_model(model_filepath)
            print(f"Successfully loaded {filename}")
            # Now you can use the loaded model
            # ... your code to use the loaded model ...
            loaded_models.append(loaded_model)
        except Exception as e:
            print(f"Error loading {filename}: {e}")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading model from: /content/drive/My Drive/P_IA/modelo_dl_5c_10ep.h5
Successfully loaded modelo_dl_5c_10ep.h5
Loading model from: /content/drive/My Drive/P_IA/modelo_rfc_5c_150ep_md20.joblib
Successfully loaded modelo_rfc_5c_150ep_md20.joblib
Loading model from: /content/drive/My Drive/P_IA/modelo_rfc_5c_150ep_md20_t15f.joblib
Successfully loaded modelo_rfc_5c_150ep_md20_t15f.joblib
Loading model from: /content/drive/My Drive/P_IA/modelo_dt_5c_md30.joblib
Successfully loaded modelo_dt_5c_md30.joblib
Loading model from: /content/drive/My Drive/P_IA/modelo_gnb_5c.joblib
Successfully loaded modelo_gnb_5c.joblib
Loading model from: /content/drive/My Drive/P_IA/modelo_svm_5c_lin.joblib
Successfully loaded modelo_svm_5c_lin.joblib
Loading model from: /content/drive/My Drive/P_IA/modelo_svm_5c_rbf.joblib
Successfully loaded modelo_svm_5c_rbf.joblib
Loading model fr

In [ ]:
#@title Feature importance (necesita RF)
estRF = os.path.join('/content/drive/My Drive/P_IA', 'modelo_rfc_5c_150ep_md20.joblib')
estRF = joblib.load(estRF)
feature_importance = pd.DataFrame({'Feature': df.columns[:-1], 'Importance': estRF.feature_importances_})
feature_importance = feature_importance.sort_values('Importance', ascending=False)
persent= 0
topfeat =5
top_features = feature_importance['Feature'].iloc[:topfeat].tolist()
print(top_features)
# X is a numpy array, convert it to a pandas DataFrame first
X_df = pd.DataFrame(X, columns=df.columns[:-1])  # Assuming X has the same columns as df except the last one
X_reduced = X_df[top_features] # Selecting columns using a list of column names is simpler with DataFrames
Xr5_test, yr5_test = X_reduced, y

['slashpath', 'Tamañospath', 'puntospath', 'puntosnetoloc', 'Tamañonetoloc']


In [ ]:
persent= 0
topfeat =15
top_features = feature_importance['Feature'].iloc[:topfeat].tolist()
print(top_features)
# X is a numpy array, convert it to a pandas DataFrame first
X_df = pd.DataFrame(X, columns=df.columns[:-1])  # Assuming X has the same columns as df except the last one
X_reduced = X_df[top_features] # Selecting columns using a list of column names is simpler with DataFrames
Xr20_test, yr20_test = X_reduced, y

['slashpath', 'Tamañospath', 'puntospath', 'puntosnetoloc', 'Tamañonetoloc', 'Tamañosscheme', 'guionpath', 'Tamañosquery', 'guionbpath', 'puntosquery', 'guionnetoloc', 'slashquery', 'guionquery', 'guionbquery', 'slashbpath']


# Calculos

In [ ]:
full_f = []
for model in loaded_models:
  try:
    tpr, tnr = evaluate_model(model, X_test, y_test)
    print(f"Model: {model}")
    print(f"True Positive Rate (TPR): {tpr}")
    print(f"True Negative Rate (TNR): {tnr}")
    print("-" * 20)
  except Exception as e:
    try:
      tpr, tnr = evaluate_model(model, Xr5_test, yr5_test)
      print(f"Model: {model}")
      print(f"True Positive Rate (TPR): {tpr}")
      print(f"True Negative Rate (TNR): {tnr}")
      print("-" * 20)
    except Exception as e:
      try:
        tpr, tnr = evaluate_model(model, Xr20_test, yr20_test)
        print(f"Model: {model}")
        print(f"True Positive Rate (TPR): {tpr}")
        print(f"True Negative Rate (TNR): {tnr}")
        print("-" * 20)
      except Exception as e:
        print(f"Error al evaluar el modelo {model}: {e}")



1/1 [==============================] - 0s 23ms/step
Error al evaluar el modelo <keras.src.engine.sequential.Sequential object at 0x78e2681deaa0>: Graph execution error:

Detected at node sequential/dense/Relu defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _ru

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomFor

Model: RandomForestClassifier(max_depth=30, n_estimators=200)
True Positive Rate (TPR): 0.05225736812464802
True Negative Rate (TNR): 0.39257751773651034
--------------------
Model: DecisionTreeClassifier(max_depth=30)
True Positive Rate (TPR): nan
True Negative Rate (TNR): 0.0
--------------------
Model: GaussianNB()
True Positive Rate (TPR): nan
True Negative Rate (TNR): 0.0
--------------------
Error al evaluar el modelo SVC(kernel='linear'): The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Tamañosquery
- Tamañosscheme
- guionbpath
- guionbquery
- guionnetoloc
- ...

Error al evaluar el modelo SVC(): The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Tamañosquery
- Tamañosscheme
- guionbpath
- guionbquery
- guionnetoloc
- ...

Error al evaluar el modelo SVC(kernel='poly'): The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Tamaños

<ipython-input-2-c7c41cfce332>:16: RuntimeWarning: invalid value encountered in scalar divide
  tpr = tp / (tp + fn)
<ipython-input-2-c7c41cfce332>:16: RuntimeWarning: invalid value encountered in scalar divide
  tpr = tp / (tp + fn)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
